### パッケージのインポート

In [48]:
import csv
import os

import pandas as pd

### 関数の定義

In [81]:
def txt_csv_converter(filename, writer):
    with open(filename) as f:
        file = f.readlines()
        for row in file:
            read = []
            for i in range(len(row)):
                read.append(row.strip(","))
            writer.writerow(read)

def get_raw_data(dir_path) -> list:
    filenames = sorted(os.listdir(dir_path))
    return filenames


In [79]:
filename_path = "../data/tsunami/NOWPHAS_Tsunami_data"
filenames = get_raw_data(filename_path+"/raw")

In [82]:
for filename in filenames:
    if filename == ".DS_Store":
        continue
    csv_file = filename_path + "/csv/" + filename[:10] + ".csv"
    with open(csv_file, mode="w", newline="") as fw:
        writer = csv.writer(fw)
        txt_csv_converter(filename_path+"/raw/"+filename, writer)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8e in position 5273: invalid start byte